In [1]:
from IPython import display
%matplotlib inline
import cv2
import gym
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys
import torch
import omegaconf
import time
import torch

import mbrl.env.cartpole_continuous as cartpole_env
import mbrl.env.reward_fns as reward_fns
import mbrl.env.termination_fns as termination_fns
import mbrl.models as models
import mbrl.planning as planning
import mbrl.util.common as common_util
import mbrl.util as util

import tactile_gym.rl_envs
from tactile_gym.sb3_helpers.params import import_parameters

%load_ext autoreload
%autoreload 2

mpl.rcParams.update({"font.size": 16})

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
# produce a display to render image
from pyvirtualdisplay import Display
_display = Display(visible=False, size=(1400, 900))
_ = _display.start()

In [3]:
# Define model working directorys
work_dir = os.path.join(os.getcwd(), 'saved_model')
print(work_dir)

/home/qt21590/Documents/Projects/tactile_gym_mbrl/mbrl-lib/notebooks/saved_model


In [4]:
# Load the environment 
env_name = 'object_push-v0'
env_kwargs_file = 'env_kwargs'
env_kwargs_dir = os.path.join(work_dir, env_kwargs_file)
env_kwargs = omegaconf.OmegaConf.load(env_kwargs_dir)

env = gym.make(env_name, **env_kwargs)
seed = 0
env.seed(seed)
rng = np.random.default_rng(seed=0)
generator = torch.Generator(device=device)
generator.manual_seed(seed)
obs_shape = env.observation_space.shape
act_shape = env.action_space.shape

argv[0]=
Loaded EGL 1.5 after reload.


pybullet build time: Mar  8 2021 17:26:24


GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3090/PCIe/SSE2
GL_VERSION=4.6.0 NVIDIA 495.29.05
GL_SHADING_LANGUAGE_VERSION=4.60 NVIDIA
Version = 4.6.0 NVIDIA 495.29.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3090/PCIe/SSE2
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [5]:
# Get cfg and agent cfg
config_file = 'cfg_dict'
config_dir = os.path.join(work_dir, config_file)
cfg = omegaconf.OmegaConf.load(config_dir)

trial_length= cfg.overrides.trial_length

agent_config_file = 'agent_cfg'
agent_config_dir = os.path.join(os.getcwd(), 'saved_model', agent_config_file)
agent_cfg = omegaconf.OmegaConf.load(agent_config_dir)

dynamics_model = common_util.create_one_dim_tr_model(cfg, obs_shape, act_shape)
model_env = models.ModelEnvPushing(env, dynamics_model, termination_fn=None, reward_fn=None, generator=generator)
replay_buffer = common_util.create_replay_buffer(cfg, obs_shape, act_shape, rng=rng)

# Create dyanmics, replay buffer and agent 
dynamics_model.load(work_dir)
replay_buffer.load(work_dir)
dynamics_model.update_normalizer(replay_buffer.get_all())  # update normalizer stats  

# Create agent 
agent = planning.create_trajectory_optim_agent_for_model(
    model_env,
    agent_cfg,
    num_particles=20
)

/home/qt21590/anaconda3/envs/tactile_gym_mbrl/lib/python3.9/site-packages/hydra/utils.py:32: UserWarning: `OmegaConf.is_none()` is deprecated, see https://github.com/omry/omegaconf/issues/547
  if OmegaConf.is_none(config):


In [6]:
# Main PETS loop
num_test_trials = 10
all_rewards = []
evaluation_result = []
plan_time = 0.0
train_time = 0.0
save_vid = True
render = True

if save_vid:
    record_every_n_frames = 1
    render_img = env.render(mode="rgb_array")
    render_img_size = (render_img.shape[1], render_img.shape[0])
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(
        os.path.join(work_dir, "evaluated_policy.mp4"),
        fourcc,
        24.0,
        render_img_size,
    )

for trial in range(num_test_trials):
    obs = env.reset()    
    agent.reset()
    
    done = False
    total_reward = 0.0
    steps_trial = 0

    evaluation_result.append(np.hstack([trial, steps_trial, obs[0:3] + env.goal_pos_workframe, obs[7:10] + env.goal_pos_workframe, env.goal_pos_workframe, 0.0, done]))

    while not done:

        # --- Doing env step using the agent and adding to model dataset ---
        start_plan_time = time.time()
        action = agent.act(obs, **{})
        next_obs, reward, done, info = env.step(action)
        plan_time = time.time() - start_plan_time

        if render:
            render_img = env.render(mode="rgb_array")
        else:
            render_img = None
        
        obs = next_obs
        total_reward += reward
        steps_trial += 1

        evaluation_result.append(np.hstack([trial, steps_trial, obs[0:3] + env.goal_pos_workframe, obs[7:10] + env.goal_pos_workframe, env.goal_pos_workframe, reward, done]))

         # use record_every_n_frames to reduce size sometimes
        if save_vid and steps_trial % record_every_n_frames == 0:

            # warning to enable rendering
            if render_img is None:
                sys.exit('Must be rendering to save video')

            render_img = cv2.cvtColor(render_img, cv2.COLOR_BGR2RGB)
            out.write(render_img)

        if steps_trial == trial_length:
            break
    
    print("Terminated at step {} with reward {}, goal reached: {}".format(steps_trial, total_reward, env.single_goal_reached))
    all_rewards.append(total_reward)

if save_vid:
    out.release()

print("The average reward over {} episodes is {}".format(num_test_trials, np.mean(all_rewards)))

# Save data 
evaluation_result = np.array(evaluation_result)
df = pd.DataFrame(evaluation_result, columns = ['trial','trial_steps','tcp_x','tcp_y','tcp_z','contact_x', 'contact_y', 'contact_z', 'goal_x', 'goal_y', 'goal_z', 'rewards', 'dones'])
pd.DataFrame(evaluation_result).to_csv(os.path.join(work_dir, "evaluation_results.csv"))

# plot evaluation results
evaluation_result_directory = os.path.join(work_dir, "evaluation_result")
if not os.path.exists(evaluation_result_directory):
    os.mkdir(evaluation_result_directory)

for trial in range(num_test_trials):
    fig, ax = plt.subplots(figsize=(12, 6))
    df.query("trial==@trial")["tcp_x"]
    ax.plot(df.query("trial==@trial")["tcp_x"], df.query("trial==@trial")["tcp_y"], "bs", label='tcp psosition')
    ax.plot(df.query("trial==@trial")["contact_x"], df.query("trial==@trial")["contact_y"], "rs", label='contact psosition')
    ax.plot(df.query("trial==@trial")["goal_x"].iloc[0], df.query("trial==@trial")["goal_y"].iloc[0], "x", markersize=20, markeredgecolor="black", label="goal position")
    ax.set_xlabel("x workframe")
    ax.set_ylabel("y workframe")
    ax.set_xlim([0, 0.3])
    ax.set_ylim([-0.1, 0.1])
    
    ax.legend()
    fig.savefig(os.path.join(evaluation_result_directory, "trial_{}.png".format(trial)))
    plt.close(fig)


Terminated at step 329 with reward -55.7151531344957, goal reached: True
Terminated at step 406 with reward -66.67548916885039, goal reached: True
Terminated at step 418 with reward -63.96934833350866, goal reached: True
Terminated at step 461 with reward -430.698516418153, goal reached: False
Terminated at step 321 with reward -53.06658449304985, goal reached: True
Terminated at step 407 with reward -62.574976947141366, goal reached: True
Terminated at step 368 with reward -62.77734973606917, goal reached: True
Terminated at step 412 with reward -73.76150616195345, goal reached: True
Terminated at step 406 with reward -66.48706880716031, goal reached: False
Terminated at step 484 with reward -462.8583261187313, goal reached: False
The average reward over 10 episodes is -139.85843193191133
